In [2]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
from scipy.interpolate import interp1d
from scipy.stats import entropy, iqr, kurtosis, mode, skew
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (Activation, BatchNormalization, Bidirectional,
                                     Conv1D, Conv2D, Dense, Dropout,
                                     Flatten, GlobalAveragePooling1D, LSTM,
                                     MaxPool1D, MaxPooling2D, Reshape,
                                     TimeDistributed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import pickle
from pandas import read_csv, unique
from scipy.signal import argrelextrema, butter, find_peaks, lfilter, lfilter_zi
from tensorflow import keras
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from kerastuner import RandomSearch, Hyperband
import kerastuner


2023-11-03 10:53:29.754897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 10:53:30.010873: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-03 10:53:30.010942: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-03 10:53:31.179320: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [9]:
X_respeck = pickle.load(open("X_respeck_resp", "rb"))
y_respeck = pickle.load(open("y_respeck_resp", "rb"))
enc = OneHotEncoder(handle_unknown='ignore')
# remove the physical activity label from y- can be left if wanted
y_respeck = [[x[0].split('|')[1]] for x in y_respeck]
enc_fitted = enc.fit(y_respeck)
enc_fitted.categories_

y_respeck = enc.transform(y_respeck).toarray()
print(enc_fitted.categories_)# One Hot is inexed as this array prints!!!!!

['lying down right|hyperventilating']
[array(['coughing', 'hyperventilating', 'normal'], dtype=object)]


In [10]:
X_train, X_temp, y_train, y_temp = train_test_split(X_respeck, y_respeck, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [42]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units1', min_value=16, max_value=256, step=16), return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
    model.add(LSTM(units=hp.Int('lstm_units2', min_value=16, max_value=256, step=16), return_sequences=True, activation='relu'))
    model.add(Conv1D(filters=hp.Int('conv1d_filters1', min_value=16, max_value=256, step=16), kernel_size=hp.Int('conv1d_kernel_size1', min_value=2, max_value=5), activation='relu', strides=2))
    model.add(MaxPool1D(pool_size=hp.Int('maxpool1d_pool_size', min_value=2, max_value=5), padding='same'))
    model.add(Conv1D(filters=hp.Int('conv1d_filters2', min_value=16, max_value=256, step=16), kernel_size=hp.Int('conv1d_kernel_size2', min_value=2, max_value=5), activation='relu', strides=1))
    model.add(GlobalAveragePooling1D())
    model.add(BatchNormalization(epsilon=1e-06))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile the model with tunable hyperparameters
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss=categorical_crossentropy,
        metrics=['categorical_accuracy']
    )
    
    return model

In [43]:
from kerastuner.tuners import BayesianOptimization

tuner = Hyperband(
    build_model,
    factor=3,  # Reduction factor for the number of models in each bracket
    max_epochs=20,  # Maximum number of training epochs for each model
    objective='categorical_accuracy',  # The metric to optimize
    directory='tuner_dir',  # Directory to store results
    project_name='Task_1_larger_lstm', # Name of the tuning project
)


# tuner = BayesianOptimization(
#     build_model,
#     objective='categorical_accuracy',  # The metric to optimize
#     max_trials=10,  # Number of trials
#     # num_initial_points=5,  # Number of initial random trials
#     directory='tuner_dir',  # Directory to store results
#     project_name='task2_larger_lstm_2'  # Name of the tuning project
# )


In [44]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), batch_size=64, shuffle=False)

Trial 10 Complete [00h 01m 24s]
categorical_accuracy: 0.5982314348220825

Best categorical_accuracy So Far: 0.623055636882782
Total elapsed time: 00h 07m 48s
INFO:tensorflow:Oracle triggered exit


In [45]:
best = tuner.get_best_models()[0]

In [46]:
best.fit(X_train, y_train, epochs=40, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/40
294/294 [==============================] - 66s 218ms/step - loss: 0.6695 - categorical_accuracy: 0.6819 - val_loss: 0.8369 - val_categorical_accuracy: 0.5347
Epoch 2/40
294/294 [==============================] - 63s 213ms/step - loss: 0.6426 - categorical_accuracy: 0.7042 - val_loss: 1.2429 - val_categorical_accuracy: 0.3357
Epoch 3/40
294/294 [==============================] - 60s 202ms/step - loss: 0.6477 - categorical_accuracy: 0.7001 - val_loss: 0.8941 - val_categorical_accuracy: 0.4248
Epoch 4/40
294/294 [==============================] - 61s 206ms/step - loss: 0.6062 - categorical_accuracy: 0.7230 - val_loss: 0.7147 - val_categorical_accuracy: 0.6758
Epoch 5/40
294/294 [==============================] - 63s 215ms/step - loss: 0.5944 - categorical_accuracy: 0.7312 - val_loss: 0.9356 - val_categorical_accuracy: 0.5658
Epoch 6/40
294/294 [==============================] - 63s 216ms/step - loss: 0.5724 - categorical_accuracy: 0.7407 - val_loss: 0.6178 - val_categorical_acc

In [47]:
with open('Task2_with_other.json', 'w') as f:
    f.write(best.to_json())
best.save('Trained_Task2_with_other.keras')

In [48]:
best.evaluate(X_test, y_test)

74/74 [==============================] - 3s 45ms/step - loss: 0.2007 - categorical_accuracy: 0.9220


[0.2006746530532837, 0.9219948649406433]

In [29]:
task1 = keras.models.load_model('Trained_Task1_2.keras')

def predict_task3(X):
    physical_action = task1.predict(X)
    physical_action = np.argmax(physical_action, axis=1)
    prediction_labels = ['ascending stairs', 'descending stairs', 'lying down back',
        'lying down on left', 'lying down on stomach', 'lying down right',
        'miscellaneous movements', 'normal walking', 'running',
        'shuffle walking', 'sitting', 'standing']
    physical_prediction = [prediction_labels[i] for i in physical_action]
    subtype_prediction = best.predict(X)
    subtype_prediction = np.argmax(subtype_prediction, axis=1)
    sub_activities = ['coughing', 'hyperventilating', 'normal']
    subtype_prediction = [sub_activities[i] for i in subtype_prediction]
    predictions = [[physical_prediction[i] + '|' + subtype_prediction[i]] for i in range(len(physical_prediction))]
    return predictions
    
pred = predict_task3(X_test)



74/74 [==============================] - 2s 22ms/step


In [32]:
def accuracy(true, pred):
    correct = 0
    for i in range(len(true)):
        print(true[i], pred[i])
        if true[i] == pred[i]:
            correct += 1
    return correct/len(true)